<a href="https://colab.research.google.com/github/AllaboyinaHariNagaMounica9/Employee-Salary-Prediction/blob/main/FlightBooking_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, HTTPException, Depends, Query
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, EmailStr, ConfigDict
from typing import Optional, List
from datetime import datetime, timedelta
from contextlib import asynccontextmanager
import asyncio
import random
import math

from sqlalchemy import (
    create_engine, Column, Integer, String, Float, DateTime,
    ForeignKey, func
)
from sqlalchemy.orm import declarative_base, relationship, sessionmaker, Session


DATABASE_URL = "sqlite:///./flights.db"

engine = create_engine(
    DATABASE_URL,
    connect_args={"check_same_thread": False} if DATABASE_URL.startswith("sqlite") else {},
    echo=False
)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()



class Airline(Base):
    __tablename__ = "Airline"
    AirlineID = Column(Integer, primary_key=True, autoincrement=True)
    AirlineName = Column(String(100), nullable=False)
    ContactEmail = Column(String(100))
    ContactNumber = Column(String(20))

    flights = relationship("Flight", back_populates="airline")


class Airport(Base):
    __tablename__ = "Airport"
    AirportID = Column(Integer, primary_key=True, autoincrement=True)
    AirportName = Column(String(150), nullable=False)
    City = Column(String(100), nullable=False)
    Country = Column(String(100), nullable=False)
    IATA_Code = Column(String(10), nullable=False)

    departures = relationship("Flight", back_populates="source_airport",
                              foreign_keys="Flight.SourceAirportID")
    arrivals = relationship("Flight", back_populates="destination_airport",
                            foreign_keys="Flight.DestinationAirportID")


class Flight(Base):
    __tablename__ = "Flight"
    FlightID = Column(Integer, primary_key=True, autoincrement=True)
    AirlineID = Column(Integer, ForeignKey("Airline.AirlineID"))
    FlightNumber = Column(String(20))
    SourceAirportID = Column(Integer, ForeignKey("Airport.AirportID"))
    DestinationAirportID = Column(Integer, ForeignKey("Airport.AirportID"))
    DepartureTime = Column(DateTime)
    ArrivalTime = Column(DateTime)
    TotalSeats = Column(Integer)
    AvailableSeats = Column(Integer)
    BaseFare = Column(Float)

    airline = relationship("Airline", back_populates="flights")
    source_airport = relationship("Airport", foreign_keys=[SourceAirportID], back_populates="departures")
    destination_airport = relationship("Airport", foreign_keys=[DestinationAirportID], back_populates="arrivals")
    bookings = relationship("Booking", back_populates="flight")
    pricings = relationship("DynamicPricing", back_populates="flight")


class User(Base):
    __tablename__ = "User"
    UserID = Column(Integer, primary_key=True, autoincrement=True)
    FullName = Column(String(100), nullable=False)
    Email = Column(String(100), unique=True, nullable=False)
    Password = Column(String(100), nullable=False)
    Phone = Column(String(20))
    Role = Column(String(20), default="User")

    bookings = relationship("Booking", back_populates="user")


class DynamicPricing(Base):
    __tablename__ = "DynamicPricing"
    PricingID = Column(Integer, primary_key=True, autoincrement=True)
    FlightID = Column(Integer, ForeignKey("Flight.FlightID"))
    Timestamp = Column(DateTime, default=datetime.utcnow)
    DemandFactor = Column(Float)
    TimeToDepartureFactor = Column(Float)
    SeatAvailabilityFactor = Column(Float)
    FinalFare = Column(Float)

    flight = relationship("Flight", back_populates="pricings")


class Booking(Base):
    __tablename__ = "Booking"
    BookingID = Column(Integer, primary_key=True, autoincrement=True)
    UserID = Column(Integer, ForeignKey("User.UserID"))
    FlightID = Column(Integer, ForeignKey("Flight.FlightID"))
    BookingDate = Column(DateTime, default=datetime.utcnow)
    TotalFare = Column(Float)
    Status = Column(String(50), default="Confirmed")

    user = relationship("User", back_populates="bookings")
    flight = relationship("Flight", back_populates="bookings")
    passengers = relationship("Passenger", back_populates="booking")


class Passenger(Base):
    __tablename__ = "Passenger"
    PassengerID = Column(Integer, primary_key=True, autoincrement=True)
    BookingID = Column(Integer, ForeignKey("Booking.BookingID"))
    PassengerName = Column(String(100))
    Age = Column(Integer)
    Gender = Column(String(10))

    booking = relationship("Booking", back_populates="passengers")



class AirlineSchema(BaseModel):
    AirlineID: Optional[int]
    AirlineName: str
    ContactEmail: Optional[EmailStr] = None
    ContactNumber: Optional[str] = None
    model_config = ConfigDict(from_attributes=True)


class AirportSchema(BaseModel):
    AirportID: Optional[int]
    AirportName: str
    City: str
    Country: str
    IATA_Code: str
    model_config = ConfigDict(from_attributes=True)


class FlightSchema(BaseModel):
    FlightID: Optional[int]
    FlightNumber: str
    AirlineID: int
    SourceAirportID: int
    DestinationAirportID: int
    DepartureTime: datetime
    ArrivalTime: datetime
    BaseFare: float
    AvailableSeats: int
    TotalSeats: int
    model_config = ConfigDict(from_attributes=True)


def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()



@asynccontextmanager
async def lifespan(app: FastAPI):

    db = SessionLocal()
    try:
        if db.query(Airline).count() == 0:
            airlines = [
                Airline(AirlineName="AirIndia", ContactEmail="info@airindia.com"),
                Airline(AirlineName="Indigo", ContactEmail="support@goindigo.in"),
                Airline(AirlineName="SpiceJet", ContactEmail="help@spicejet.com"),
            ]
            db.add_all(airlines)
            db.commit()

        if db.query(Airport).count() == 0:
            airports = [
                Airport(AirportName="Chhatrapati Shivaji Maharaj International", City="Mumbai", Country="India", IATA_Code="BOM"),
                Airport(AirportName="Indira Gandhi International", City="New Delhi", Country="India", IATA_Code="DEL"),
                Airport(AirportName="Kempegowda International", City="Bengaluru", Country="India", IATA_Code="BLR"),
            ]
            db.add_all(airports)
            db.commit()
    finally:
        db.close()

    yield


app = FastAPI(title="Infosys Flight Management API", lifespan=lifespan)


app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.get("/flights", response_model=List[FlightSchema])
def get_all_flights(db: Session = Depends(get_db)):
    return db.query(Flight).all()


@app.post("/airlines", response_model=AirlineSchema)
def create_airline(airline: AirlineSchema, db: Session = Depends(get_db)):
    new_airline = Airline(**airline.model_dump())
    db.add(new_airline)
    db.commit()
    db.refresh(new_airline)
    return new_airline


@app.get("/airports", response_model=List[AirportSchema])
def list_airports(db: Session = Depends(get_db)):
    return db.query(Airport).all()